In [16]:
import pandas as pd
from time import time
from math import ceil
import scipy.sparse as sp
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, RepeatedStratifiedKFold, StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix,roc_curve, roc_auc_score, precision_score, recall_score, precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.decomposition import TruncatedSVD
#from sklearn.model_selection import cross_validation
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.naive_bayes import BernoulliNB
import imblearn.under_sampling as us

import numpy as np

In [19]:
train_data = pd.read_table('train_data_drug.txt',header=None,skiprows=0)
test_data = pd.read_table('test_data_drug.txt',header=None,skiprows=0)

In [20]:
print(type(train_data[0][1]))

<class 'numpy.int64'>


In [21]:
def dense2sparse(records):

    data = []
    i = []
    j = []

    for index, record in enumerate(records):
        features = map(int, record.split())
        for feature in features:
            data.append(1)
            i.append(index)
            j.append(feature - 1)
    return sp.coo_matrix((data, (i,j)), shape = [records.size, 100000]).tocsr()

#%%

In [22]:
train = train_data[1]
test = test_data[0]

# create sparse representations of the data|
sparse_train = dense2sparse(train)
sparse_test = dense2sparse(test)

In [ ]:
print(sparse_train)

In [ ]:
arrayyy = sparse_train.toarray()
print(arrayyy)
label = np.array(train_data[0])

In [25]:
sparsity = 1.0 - (np.count_nonzero(arrayyy) / arrayyy.size)
print('The sparsity of X is ', sparsity )

The sparsity of X is  0.990903


In [ ]:
#lets check again how csr look for the converted array

S1 = sp.csr_matrix(arrayyy)
print(S1)

Feature selection using chi2

In [200]:
#k_features = 225
ch2a = SelectKBest(score_func=chi2, k=100)
train_ch2 = ch2a.fit_transform(sparse_train.toarray(), np.array(train_data[0]))
test_ch2 = ch2a.transform(sparse_test.toarray())

Feature selection using svd

In [201]:

svd_data = TruncatedSVD(n_components=100, random_state=27, n_iter = 30)
train_SVD = svd_data.fit_transform(sparse_train.toarray(), np.array(train_data[0]))
test_SVD = svd_data.transform(sparse_test.toarray())
pd.DataFrame(train_SVD).head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,4.209650,-0.733033,-1.570471,-0.766227,-0.393375,0.175806,0.264959,0.055949,0.468206,-0.161636,...,-0.262286,0.113342,-0.166893,-1.389635,-0.353081,-0.825687,-0.133248,0.298443,-0.625724,-0.053095
1,3.860958,-0.456128,-1.631214,-1.053896,-0.212739,-0.033123,0.278416,-0.053522,0.025920,-0.045390,...,0.409617,0.665554,-0.261859,0.417001,-0.673778,-0.184718,-0.112747,-0.748529,0.208285,-0.126590
2,3.767744,-0.569020,-1.656236,-0.999111,-0.223673,0.103723,0.485975,0.119731,0.177655,-0.139602,...,0.001142,-0.192767,-0.116757,0.718297,0.364157,0.308691,-0.226423,0.027557,-0.135113,0.176790
3,3.895828,-0.485519,-1.733733,-0.907990,-0.307748,-0.088855,0.671020,-0.141903,0.011077,0.076628,...,0.288983,-0.082363,-0.575769,0.253564,0.222212,0.275303,0.653129,-0.442632,0.102576,-0.659107
4,3.881690,-0.209864,-1.515431,-0.900986,-0.206139,0.018264,0.451893,0.097388,0.044032,-0.103366,...,0.313554,-0.641057,-0.627448,0.615780,-0.466243,-0.755983,-0.306418,0.196033,0.272390,-0.142676


UnderSampling and OverSampling

In [99]:
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler

oversample = RandomOverSampler(sampling_strategy='minority')
# fit and apply the transform
train_oversample, lable_oversample = oversample.fit_resample(train_ch2, np.array(train_data[0]))
train_oversample = sp.csr_matrix(train_oversample)

In [178]:
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler

oversample = RandomOverSampler(sampling_strategy='minority')
# fit and apply the transform
train_oversample, lable_oversample = oversample.fit_resample(train_SVD, np.array(train_data[0]))
train_oversample = sp.csr_matrix(train_oversample)

In [202]:
from pandas.core.common import random_state
#Use undersampling algorithm
import imblearn.under_sampling as us
# enn = us.EditedNearestNeighbours(random_state=ceil(time())
# train_res = enn.fit_sample(reduced_train, label)
# train_res = sp.csr_matrix(train_res)

from imblearn.under_sampling import RandomUnderSampler
rosUnder = RandomUnderSampler(random_state=ceil(time()))
train_Under, label_Under = rosUnder.fit_resample(train_ch2, np.array(train_data[0]))
#print(sorted(Counter(label_resampledUnder).items()),label_resampledUnder.shape)
train_Under = sp.csr_matrix(train_Under)

In [210]:
from pandas.core.common import random_state
#Use undersampling algorithm
import imblearn.under_sampling as us
# enn = us.EditedNearestNeighbours(random_state=ceil(time())
# train_res = enn.fit_sample(reduced_train, label)
# train_res = sp.csr_matrix(train_res)

from imblearn.under_sampling import RandomUnderSampler
rosUnder = RandomUnderSampler(random_state=ceil(time()))
train_Under, label_Under = rosUnder.fit_resample(train_SVD, np.array(train_data[0]))
#print(sorted(Counter(label_resampledUnder).items()),label_resampledUnder.shape)
train_Under = sp.csr_matrix(train_Under)

Split the data

In [112]:
X_train, X_test, y_train, y_test = train_test_split(train_oversample, lable_oversample, test_size=0.2, random_state=13)

In [211]:
X_train_under, X_test_under, y_train_under, y_test_under = train_test_split(train_Under, label_Under, test_size=0.2, random_state=13)

KNN classifier

In [212]:
KNN = KNeighborsClassifier(n_neighbors = 31)
KNN.fit(X_train_under, y_train_under)
y_pred_knn = KNN.predict(X_test_under)
#print(y_pred_knn)
#print('Accuracy Score: ',metrics.accuracy_score(y_test,y_pred)*100,'%',sep='')
#print('Confusion Matrix: ',metrics.confusion_matrix(y_test,y_pred), sep = '\n')

Decision Tree

In [213]:
#clf_gini = DecisionTreeClassifier(criterion='gini', max_depth=3, random_state=0)

clf_gini = DecisionTreeClassifier(class_weight={0: 1, 1: 2})
# fit the model
clf_gini.fit(X_train_under, y_train_under)
y_pred_gini = clf_gini.predict(X_test_under)
y_pred_gini_test = clf_gini.predict(test_SVD)
print(y_pred_gini)

[1 1 0 1 1 1 1 1 0 0 0 0 1 1 0 1 1 0 0 1 1 1 0 1 0 1 1 0 1 1 1 0]


In [174]:
#take in a .txt file
np.savetxt(r'format.txt', y_pred_gini_test, fmt='%s')

Logistic regression

In [214]:
# define model
w = {0:1, 1:3}
lg1 = LogisticRegression(random_state=13, class_weight=w)
# fit it
#lg1.fit(X_train,y_train)
lg1.fit(X_train_under,y_train_under)
# test
#y_pred = lg1.predict(X_test)
y_pred_log = lg1.predict(X_test_under)
#y_pred_log = lg1.predict(test_SVD)
# performance
#print(f'Accuracy Score: {accuracy_score(y_test,y_pred)}')
#print(f'Confusion Matrix: \n{confusion_matrix(y_test, y_pred)}')
#print(f'Area Under Curve: {roc_auc_score(y_test, y_pred)}')
#print(f'Recall score: {recall_score(y_test,y_pred)}')

Naive Bayes

In [215]:
bnb = BernoulliNB(alpha=.75)
bnb.fit(X_train_under, y_train_under)
results_NM = bnb.predict(X_test_under)
#results_NM_TEST = bnb.predict(test_SVD)


In [ ]:
Cross validation

In [216]:
# Shuffling data for cross validation

permutation = np.arange(label.size)
np.random.shuffle(permutation)

shuffled_train = X_train[permutation]
shuffled_label = y_train[permutation]

# cross-validation
scores_kbest = cross_val_score(bnb, shuffled_train, shuffled_label, cv=10, scoring='f1')

scores_knn = cross_val_score(KNN, shuffled_train, shuffled_label, cv=10, scoring='f1')

scores_tree = cross_val_score(clf_gini, shuffled_train, shuffled_label, cv=10, scoring='f1')

scores_logis = cross_val_score(lg1, shuffled_train, shuffled_label, cv=10, scoring='f1')

print(str(scores_kbest.mean(0)))
print(str(scores_knn.mean(0)))
print(str(scores_tree.mean(0)))
print(str(scores_logis.mean(0)))


0.8918504922841584
0.42962975611144316
0.9547225444311497
0.992621588431413


In [199]:
np.savetxt(r'format.txt', y_pred_log, fmt='%s')

In [ ]:
F1 and recall matrix

In [217]:
print("Recall: Decision Tree: ",recall_score(y_test_under, y_pred_gini))
print("F1: Decision Tree: ",f1_score(y_test_under, y_pred_gini))

print("Recall: KNN: ",recall_score(y_test_under, y_pred_knn))
print("F1: KNN: ",f1_score(y_test_under, y_pred_knn))

print("Recall: Logistic Regression: ",recall_score(y_test_under, y_pred_log))
print("F1: Logistic regression: ",f1_score(y_test_under, y_pred_log))

print("Recall: BNB: ",recall_score(y_test_under, results_NM))
print("F1: BNB: ",f1_score(y_test_under, results_NM))

Recall: Decision Tree:  0.8823529411764706
F1: Decision Tree:  0.8108108108108107
Recall: KNN:  0.47058823529411764
F1: KNN:  0.5925925925925927
F1: Logistic regression:  0.7222222222222222
Recall: BNB:  0.7647058823529411
F1: BNB:  0.7647058823529412
